In [1]:
from typing import Tuple
import numpy as np
import torch
import torch.utils.data as data
from torch.autograd import Variable
from net import HGCTNet

In [2]:
# model = HGCTNet(num_classes=2, num_hid=16, depth=2, num_heads=2, hga=True, trans=True)
# sigs = Variable(torch.randn(5, 3, 12000))
# feas = Variable(torch.randn(5, 53))
# estbp = model(sigs, feas)

In [3]:
def create_dataset(dataset: list, window_size: int) -> Tuple[torch.Tensor]:
    """
    Transform a time series into a prediction dataset.
    """
    feature_list, signal_list, bp_list = [], [], []
    for i in range(window_size, len(dataset)):
        sample = dataset[i - window_size: i]
        signal, feature, bp = list(zip(*sample))
        signal_list.append(np.stack(signal, axis=-1, dtype=np.float32))
        feature_list.append(np.stack(feature, axis=-1, dtype=np.float32))
        bp_list.append(np.stack(bp, axis=-1, dtype=np.float32))
    return torch.Tensor(np.array(signal_list)), torch.Tensor(np.array(feature_list)), torch.Tensor(np.array(bp_list))

In [4]:
train_signal = torch.load('train_signal.pt')[:-3].numpy()
train_feature = torch.load('train_feature.pt')[:-3].numpy()
train_bp = torch.load('train_bp.pt').numpy()
train_dataset = list(zip(train_signal, train_feature, train_bp))

test_signal = torch.load('test_signal.pt').numpy()
test_feature = torch.load('test_feature.pt').numpy()
test_bp = torch.load('test_bp.pt')[:-3].numpy()
test_dataset = list(zip(test_signal, test_feature, test_bp))

In [5]:
print(train_feature.shape)
print(train_signal.shape)
print(train_bp.shape)

print(test_feature.shape)
print(test_signal.shape)
print(test_bp.shape)

(2700, 31)
(2700, 3, 24804)
(2700, 2)
(810, 31)
(810, 3, 24804)
(810, 2)


In [6]:
# train_feature, train_signal, train_bp = create_dataset(train_dataset, 100)
valid_feature, valid_signal, valid_bp = create_dataset(test_dataset, 100)
# train_loader = data.DataLoader(data.TensorDataset(train_feature, train_signal, train_bp), shuffle=True, batch_size=64, num_workers=0)
valid_loader = data.DataLoader(data.TensorDataset(valid_feature, valid_signal, valid_bp), shuffle=True, batch_size=64, num_workers=0)

In [7]:
for idx, (signal, feature, bp) in enumerate(valid_loader):
    print(signal.shape)
    print(feature.shape)
    print(bp.shape)
    break

torch.Size([64, 3, 24804, 100])
torch.Size([64, 31, 100])
torch.Size([64, 2, 100])


In [8]:
model = HGCTNet(num_classes=2, num_hid=16, depth=2, num_heads=2, hga=True, trans=True)